# Interfaces, Patterns, and Modularity

Design patterns:

* Are reusable solutions to many common design problems appearing in software engineering. 

* Are often language-agnostic and thus can be expressed using many programming languages.


### zope.interface

Consider the following code that checks if a list of geometric figures collide with the Axis-Aligned Bounding Box (AABB) algorithm. A simple implementation would be as follows:


In [1]:
!python _01_zope_interface/colliders_simple.py


(Square(x=0, y=0, size=10), Rect(x=5, y=5, width=20, height=20))
(Square(x=0, y=0, size=10), Circle(x=1, y=1, radius=2))
(Rect(x=5, y=5, width=20, height=20), Square(x=15, y=20, size=5))


But what if we add another figure that is missing some important attribute?. We'll only see the error at runtime:


In [2]:
from _01_zope_interface.colliders_simple import Point, Square, find_collisions

try:
    coll_point = find_collisions([
        Square(0, 0, 10),
        Point(100, 200),
    ])
except AttributeError as e:
    print(f'ERROR: {repr(e)}')

ERROR: AttributeError("'Point' object has no attribute 'bounding_box'")


We can avoid those kind of errors declaring explicit interfaces. The zope.interface allows us to create and implement explicit interfaces, as follows:

In [3]:
!python _01_zope_interface/colliders_interfaces.py


Valid attempt:

(Square(x=0, y=0, size=10), Rect(x=5, y=5, width=20, height=20))
(Square(x=0, y=0, size=10), Circle(x=1, y=1, radius=2))
(Rect(x=5, y=5, width=20, height=20), Square(x=15, y=20, size=5))

Invalid attempt (a detailed exception will be raised):

Traceback (most recent call last):
  File "/home/work/Documents/Github/will-i-amv-books/Expert-Python-Programming-Fourth-Edition/Chapter_5/_01_zope_interface/colliders_interfaces.py", line 106, in <module>
    for collision in find_collisions([
  File "/home/work/Documents/Github/will-i-amv-books/Expert-Python-Programming-Fourth-Edition/Chapter_5/_01_zope_interface/colliders_interfaces.py", line 27, in find_collisions
    verifyObject(ICollidable, item)
  File "/home/work/.cache/pypoetry/virtualenvs/expert-python-programming-fourth-edition-em7utuy--py3.10/lib/python3.10/site-packages/zope/interface/verify.py", line 166, in verifyObject
    return _verify(iface, candidate, tentative, vtype='o')
  File "/home/work/.cache/pypoetry/vi

In the last code example, the `verifyObject` function (there's also a similar `verifyClass` function) is used to raise a detailed exception if one of the class instances does not implement the interface `ICollidable` correctly.


The `verifyClass` and `verifyObject` functions only verify the surface area of the interface and aren't able to traverse into attribute types. 

We can do a more in-depth verification with the `validateInvariants` function, which checks if the class has all attributes necessary to satisfy the `ICollidable` interface, but also verifies whether the structure of those attributes satisfies constraints defined in the interface.

The new example is as follows:

In [4]:
!python _01_zope_interface/colliders_invariants.py


Valid attempt:

(Square(x=0, y=0, size=10), Rect(x=5, y=5, width=20, height=20))
(Square(x=0, y=0, size=10), Circle(x=1, y=1, radius=2))
(Rect(x=5, y=5, width=20, height=20), Square(x=15, y=20, size=5))

Invalid attempt (a detailed exception will be raised):

Traceback (most recent call last):
  File "/home/work/Documents/Github/will-i-amv-books/Expert-Python-Programming-Fourth-Edition/Chapter_5/_01_zope_interface/colliders_invariants.py", line 120, in <module>
    for collision in find_collisions([
  File "/home/work/Documents/Github/will-i-amv-books/Expert-Python-Programming-Fourth-Edition/Chapter_5/_01_zope_interface/colliders_invariants.py", line 27, in find_collisions
    ICollidable.validateInvariants(item)
  File "/home/work/.cache/pypoetry/virtualenvs/expert-python-programming-fourth-edition-em7utuy--py3.10/lib/python3.10/site-packages/zope/interface/interface.py", line 869, in validateInvariants
    invariant(obj)
  File "/home/work/Documents/Github/will-i-amv-books/Expert-Pyt

In the last code example, the `Point` class seems to implement the `ICollidable` interface on the surface, bit it does not satisfy the interface invariant: it doesn't have the `bounding_box` property as instance of the `Box` type, so an exception is raised.


### Abstract Base Classes (ABCs)

In [5]:
some_type = []


In Python, type comparisons like the following example should be avoided:


In [6]:
assert type(some_type) == list


A better approach is to use the isinstance() function, so you can do type checking against a given type and also its subtypes, as follows: 



In [7]:
assert isinstance(some_type, list)


You can even use a range of types to check the type compatibility, as follows:


In [8]:
assert isinstance(some_type, (list, tuple, range))




But this method won't work if you create a custom class that defines the `__iter__()` method (so it behaves similar to a list) but inherits directly from `object`. This is where ABCs are useful.

ABC is a class that doesn't provide a concrete implementation, but defines a blueprint of a class that may be used to check against type compatibility.


The following shows a generic ABC, many incorrect implementations and a correct implementation of it:

In [9]:
!python _02_function_annotations_ABC/dummy_interface.py



Instantiating ABC: <class '__main__.DummyInterface'>
ERROR: TypeError("Can't instantiate abstract class DummyInterface with abstract methods dummy_method, dummy_property")

Instantiating ABC: <class '__main__.InvalidDummy'>
ERROR: TypeError("Can't instantiate abstract class InvalidDummy with abstract methods dummy_method, dummy_property")

Instantiating ABC: <class '__main__.MissingMethodDummy'>
ERROR: TypeError("Can't instantiate abstract class MissingMethodDummy with abstract method dummy_method")

Instantiating ABC: <class '__main__.MissingPropertyDummy'>
ERROR: TypeError("Can't instantiate abstract class MissingPropertyDummy with abstract method dummy_property")

Instantiating ABC: <class '__main__.Dummy'>
SUCCESS


From the last code examples:

* The `@abstractmethod` decorator indicates that a method must be implemented in classes that subclass the ABC. 

* If a class doesn't implement one of the abstract methods, a `TypeError` exception will be raised.

* This way you can ensure implementation completeness.

We can modify the collider detection code to use ABCs, as follows:


In [10]:
!python _02_function_annotations_ABC/colliders_abc.py


Valid attempt:

(Square(x=0, y=0, size=10), Rect(x=5, y=5, width=20, height=20))
(Square(x=0, y=0, size=10), Circle(x=1, y=1, radius=2))
(Rect(x=5, y=5, width=20, height=20), Square(x=15, y=20, size=5))

Invalid attempt (with a dataclass that doesn't inherit from the ABC):

TypeError('Point(x=100, y=200) is not a collider')

Invalid attempt (with a dataclass that doesn't implement the ABC's abstract methods):

TypeError("Can't instantiate abstract class PointWithABC with abstract method bounding_box")


ABCs provide the special `__subclasshook__(cls)` method. We can use it to add validation logic to check whether a subclass of `ColliderABC` is implicitly compatible with it, as follows:


In [11]:
!python _02_function_annotations_ABC/colliders_subclasshooks.py


Valid attempt:

(Square(x=0, y=0, size=10), Rect(x=5, y=5, width=20, height=20))
(Square(x=0, y=0, size=10), Circle(x=1, y=1, radius=2))
(Rect(x=5, y=5, width=20, height=20), Square(x=15, y=20, size=5))

Invalid attempt:

Traceback (most recent call last):
  File "/home/work/Documents/Github/will-i-amv-books/Expert-Python-Programming-Fourth-Edition/Chapter_5/_02_function_annotations_ABC/colliders_subclasshooks.py", line 127, in <module>
    for collision in find_collisions([
  File "/home/work/Documents/Github/will-i-amv-books/Expert-Python-Programming-Fourth-Edition/Chapter_5/_02_function_annotations_ABC/colliders_subclasshooks.py", line 39, in find_collisions
    raise TypeError(f"{item} is not a collider")
TypeError: Point(x=100, y=200) is not a collider


With the `__subclasshook__()` method defined that way, `ColliderABC` becomes an implicit interface:

* Any object will be considered an instance of ColliderABC as long as it has the structure that passes the subclass hook check.

* So we can create classes compatible with the `ColliderABC` interface without explicitly inheriting from it.

The following example shows implicit interfaces in action:

In [12]:
from _02_function_annotations_ABC.colliders_subclasshooks import ColliderABC, Line, Point


line = Line(Point(0, 0), Point(100, 100))


In [13]:
line.bounding_box

Box(x1=0, y1=0, x2=100, y2=100)

In [14]:
isinstance(line, ColliderABC)

True

### Interfaces with typing.Protocol

Support for typing in the standard library and popular-third party projects grew greatly in recent years:

* We can use type annotations to perform structural subtyping (static duck-typing) with the typing.Protocol type. 

* By subclassing this type, we create a definition of our interface. 

* We can also apply simple runtime checks similar to ABC subclass hooks.


We can modify the collider detection code to use Protocols, as follows:


In [15]:
!python _03_Interfaces_type_annotations/colliders_protocol.py


Valid attempt:

(Square(x=0, y=0, size=10), Rect(x=5, y=5, width=20, height=20))
(Square(x=0, y=0, size=10), Circle(x=1, y=1, radius=2))
(Rect(x=5, y=5, width=20, height=20), Square(x=15, y=20, size=5))

Invalid attempt:

Traceback (most recent call last):
  File "/home/work/Documents/Github/will-i-amv-books/Expert-Python-Programming-Fourth-Edition/Chapter_5/_03_Interfaces_type_annotations/colliders_protocol.py", line 128, in <module>
    for collision in find_collisions([
  File "/home/work/Documents/Github/will-i-amv-books/Expert-Python-Programming-Fourth-Edition/Chapter_5/_03_Interfaces_type_annotations/colliders_protocol.py", line 40, in find_collisions
    raise TypeError(f"{item} is not a collider")
TypeError: Point(x=100, y=200) is not a collider


### Inversion of control (IoC)

* A traditional architecture of a program consists of a layered structure of procedures, where control always goes from top to bottom (higher-level layers invoke procedures from lower layers)

* For example, control is passed from application to library functions, which pass it deeper to lower-level libraries and, eventually, return it back to the application.

* IoC happens when a library passes control up to the application so that the application can take part in the library behavior.


For example, consider sorting a list of integer numbers with Python's `sorted` function:


In [4]:
list_ = [3, 6, 2, 1, 4, 5]

In [5]:
sorted(list_)

[1, 2, 3, 4, 5, 6]

But with the `sorted` funcion, we can sort the numbers in another ways. For instance, we can sort numbers by the absolute distance from number 3, as follows:


In [7]:
def distance_from_3(x):
    return abs(x - 3)


sorted(list_, key=distance_from_3)

[3, 2, 4, 1, 5, 6]

Here is where IoC happens. 
* The `sorted` function "upcalls" back to the `distance_from_3` function provided by the application as argument. 

* Now it's a library that calls the functions from the application, and thus the flow of control is reversed.

* Callback-based IoC is also humorously referred to as the Hollywood principle ("don't call us, we'll call you").


Some more examples of IoC:

* Polymorphism: When a custom class inherits from a base class and base methods are supposed to call custom methods

* Argument passing: When the receiving function is supposed to call methods of the supplied object

* Decorators: When a decorator function calls a decorated function

* Closures: When a nested function calls a function outside of its scope


### Inversion of control in applications

Consider a small web app that can track web page views using so-called tracking pixels and serve page view statistics over HTTP endpoints. The 2 endpoints are:

* /track: Returns an HTTP response with a 1x1 pixel GIF image. Upon request, it will store the Referer header and increase the number of requests associated with that value.

* /stats: Reads the top 10 most common Referer values received on the track/ endpoint and returns an HTTP response containing a summary of the results in JSON format.


The following shows an inplementation of that webapp using Flask. Execute the following command in another terminal to start the server: 

In [8]:
# !python _04_Inversion_of_control/tracking.py


The current implementation stores request counters in memory. Whenever the application is restarted, the existing counter values will be lost. 

To keep the data between restarts, we'll replace our storage implementation. We could use:

* A text file.

* The built-in `shelve` module

* A relational database management system (RDBMS) like `PostgreSQL`.

* An in-memory key-value (K-V) or data structure storage like `Redis`.


If we don't know yet which storage to use, we can make it pluggable so we can switch storage backends depending on our needs. 

* To do so, we will have to invert the flow of control in our `track()` and `stats()` view functions.

* We'll define our views storage interface as an ABC `ViewsStorageBackend` with the following methods:

    - `increment()` to increase the counter value by one

    - `most_common()` to retrieve the 10 most often requested keys

* We'll provide 2 implementations of that class:

    - `CounterBackend` using a `Counter` as in-memory collection.

    - `RedisBackend`  using a `Redis` as in-memory K-V store.

* Now we need to invert control of our `track()` and `stats()` functions in a way that will allow us to plug-in a different storage implementation. We do it in 2 steps.

* First, we add an extra `storage` argument of `ViewsStorageBackend` type to the view functions: 
    
    - We can switch the implementation of storage for different classes with a compatible interface. 

    - Also, we can unit-test the view functions in isolation from storage implementations.

* Next, we do the route registration: 

    - We can't use the `@app.route()` decorator directly on the view functions, because Flask won't be able to resolve the storage argument on its own. 

    - Instead, we "pre-inject" the desired storage implementations into view functions using functools.partial, then we register them with the `app.route()` function call.


The following is an application configuration that uses Redis as a storage backend. 

For this example you need to install `docker` and `docker-compose`. Execute the following command in another terminal to start the server:


In [ ]:
#!docker compose -f _05_Inversion_of_control_p2/docker-compose.yml up


### Dependency Injection